In [10]:
import cv2
import csv
import math
import numpy as np
import os
from datetime import datetime
from pathlib import Path
import mediapipe as mp

# Инициализация модели MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    min_detection_confidence=0.5
)

# Конфигурация
CSV_FILENAME = "результаты_анализа.csv"
SUPPORTED_EXT = ('.jpg', '.jpeg', '.png', '.webp')

# Ключевые точки для анализа
LANDMARK_IDS = {
    'forehead': [103, 332, 333, 298],
    'left_brow': [70, 63, 105, 66, 107],
    'right_brow': [336, 296, 334, 293, 300],
    'head': [10, 338, 297, 332, 284, 251]
}

def calculate_ratio(p1, p2, img_shape):
    """Вычисление расстояния между точками"""
    h, w = img_shape[:2]
    return math.hypot((p1.x - p2.x)*w, (p1.y - p2.y)*h)

def process_image(img_path):
    """Обработка одного изображения"""
    result = {
        'имя_файла': img_path.name,
        'дата_обработки': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'форма_лба': 'не определено',
        'положение_бровей': 'не определено',
        'форма_головы': 'не определено',
        'ошибка': None
    }

    try:
        # Чтение файла с русским названием
        img = cv2.imdecode(np.fromfile(img_path, dtype=np.uint8), cv2.IMREAD_COLOR)
        if img is None:
            raise ValueError("Не удалось прочитать файл")

        # Поиск ключевых точек
        results = face_mesh.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if not results.multi_face_landmarks:
            raise ValueError("Лицо не обнаружено")

        landmarks = results.multi_face_landmarks[0].landmark
        h, w = img.shape[:2]

        # Анализ лба
        forehead_top = calculate_ratio(landmarks[103], landmarks[332], (h, w))
        forehead_bottom = calculate_ratio(landmarks[333], landmarks[298], (h, w))
        result['соотношение_лба'] = round(forehead_top / forehead_bottom, 2)
        result['форма_лба'] = "суженный" if result['соотношение_лба'] < 0.95 else "прямоугольный"

        # Анализ бровей
        brow_heights = []
        for side in ['left_brow', 'right_brow']:
            brow_y = np.mean([landmarks[i].y for i in LANDMARK_IDS[side]]) * h
            eye_y = np.mean([landmarks[i].y for i in [145, 159] if side == 'left_brow' or [374, 386]]) * h
            brow_heights.append(eye_y - brow_y)
        
        result['средняя_высота_бровей'] = round(np.mean(brow_heights), 1)
        result['положение_бровей'] = "высокие" if result['средняя_высота_бровей'] > 15 else "низкие"

        # Анализ головы
        head_width = calculate_ratio(landmarks[10], landmarks[332], (h, w))
        head_height = calculate_ratio(landmarks[338], landmarks[284], (h, w))
        result['соотношение_головы'] = round(head_width / head_height, 2)
        result['форма_головы'] = "вертикальная" if result['соотношение_головы'] < 0.65 else "горизонтальная"

    except Exception as e:
        result['ошибка'] = str(e)

    return result

def save_results(data, filename):
    """Сохранение результатов в CSV"""
    file_exists = os.path.exists(filename)
    with open(filename, 'a', newline='', encoding='utf-8-sig') as f:
        writer = csv.DictWriter(f, fieldnames=data.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(data)

def process_folder(folder_path):
    """Обработка всех изображений в папке"""
    folder = Path(folder_path)
    if not folder.exists():
        print(f"Ошибка: Папка {folder} не существует")
        return

    # Поиск файлов изображений
    image_files = []
    for ext in SUPPORTED_EXT:
        image_files.extend(folder.glob(f'*{ext}'))
        image_files.extend(folder.glob(f'*{ext.upper()}'))

    if not image_files:
        print("Не найдено подходящих файлов")
        return

    # Обработка файлов
    for img_path in image_files:
        print(f"Обработка: {img_path.name}")
        result = process_image(img_path)
        save_results(result, CSV_FILENAME)

    print(f"\nГотово! Результаты сохранены в {CSV_FILENAME}")

if __name__ == "__main__":
    target_folder = input("Введите путь к папке с фотографиями: ").strip()
    process_folder(target_folder)

Обработка: -_OnqZ0eJas.jpg
Обработка: 1200а.jpg
Обработка: 16.jpg
Обработка: 16028.jpg
Обработка: 16503а (1).jpg
Обработка: 16503а (2).jpg
Обработка: 18474.jpg
Обработка: 1mvbji9E8j8.jpg
Обработка: 20729.jpg
Обработка: 21.jpg
Обработка: 2510.jpg
Обработка: 2656b.jpg
Обработка: 2gKSVLBc23E.jpg
Обработка: 2Si3ZA8zLRo.jpg
Обработка: 31447а (1).jpg
Обработка: 31447а (2).jpg
Обработка: 42892.jpg
Обработка: 43BhsyTyaTk.jpg
Обработка: 4Fybt5QsBnM.jpg
Обработка: 4vg79ZoHb-4.jpg
Обработка: 5199.jpg
Обработка: 67CHcbI07YE.jpg
Обработка: 6Qxws_0mXf4.jpg
Обработка: 70301.jpg
Обработка: 8201а (1).jpg
Обработка: 8201а (2).jpg
Обработка: 9r7B-zSbmuE.jpg
Обработка: a5H7iysrPGg.jpg
Обработка: B9nEbJGDY6M.jpg
Обработка: brittney griner basketball1.jpg
Обработка: brittney griner basketball1a.jpg
Обработка: brittney griner basketball1s.jpg
Обработка: brittney griner basketball2.jpg
Обработка: brittney griner basketball3.jpg
Обработка: brittney griner basketball4.jpg
Обработка: brittney griner basketball5.